# DeepState - Experiments

- Using DeepState with gluonts to run some experiments
- Follow this installation guide: http://gluon-ts-staging.s3-accelerate.dualstack.amazonaws.com/PR-271/4/install.html

Important note! Some of these models are not yet released in version "0.3.3" which is currently the most recent version on pypi from where you install packages via `pip install`. Hence, you should follow **Install from Source Code** (which I did) **Install from Github**

### Deep State Space Models (Rangapuram et al., 2018)

**DeepState** combines state space models (SSM) with a recurrent neural network (RNN). The SSM is applied *locally* to the individual time series that is parametrized using a *global* representation in form of a RNN. The RNN is trained on the entire dataset. 

- [`DeepStateEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deepstate.html#gluonts.model.deepstate.DeepStateEstimator)


In [1]:
# if version does not include DeepState, uncomment the following:
!pip install git+https://github.com/awslabs/gluon-ts.git

import gluonts
print(gluonts.__version__)

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports 
from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

  Cloning https://github.com/awslabs/gluon-ts.git to /tmp/pip-req-build-y2sffxww
  Installing build dependencies ... done
    100% |████████████████████████████████| 81kB 1.9MB/s ta 0:00:011
    100% |████████████████████████████████| 28.4MB 1.9MB/s  eta 0:00:01
    100% |████████████████████████████████| 10.4MB 6.3MB/s eta 0:00:011
    100% |████████████████████████████████| 4.8MB 8.0MB/s eta 0:00:011
    100% |████████████████████████████████| 61kB 47.9MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 68.2MB/s eta 0:00:01
  Running setup.py bdist_wheel for gluonts ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-u57oaodr/wheels/09/ba/b9/c830628df4d977a9dfe661f6001cdb861fc3f2524bac0ee433
  Running setup.py bdist_wheel for holidays ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/40/a9/2b/94ac5464363d37564a87dc93a9d21a5850aac14a4608197003
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2

INFO:root:Using CPU


In [2]:
def deep_state(seed=42, data="m4_daily", epochs=25, batches=50):
    dataset = get_dataset(data)

    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
#         ctx=mx.gpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepStateEstimator(
        trainer=trainer,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         past_length=48,
        use_feat_static_cat=True,
        cardinality=[cardinality],
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}

    pprint(output)

# if __name__ == "__main__":
#     deep_state()

In [ ]:
%%time
if __name__ == "__main__":
    deep_state(seed=42, data="m4_daily", epochs=25, batches=50)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [06:08<00:00,  7.38s/it, avg_epoch_loss=-1.4]    
INFO:root:Epoch[0] Elapsed time 368.850 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-1.396312
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [01:57<00:00,  2.35s/it, avg_epoch_loss=0.0665]
INFO:root:Epoch[1] Elapsed time 117.448 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.066499
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [01:56<00:00,  2.32s/it, avg_epoch_loss=-1.24]
INFO:root:Epoch[2] Elapsed time 116.127 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-1.238061
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [01:57<00:00,  2.34s/it, avg_epoch_loss=-1.52]
INFO:root:Epoch[3] Elapsed time 117.205 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=-1.518090
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [01:58<00:00,  2.37s/

In [5]:
%%time
if __name__ == "__main__":
    deep_state(seed=42, data="m4_daily", epochs=35, batches=50)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training


MXNetError: [09:48:34] src/ndarray/ndarray.cc:1279: GPU is not enabled

Stack trace returned 10 entries:
[bt] (0) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x23d55a) [0x7ff601e4c55a]
[bt] (1) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x23dbc1) [0x7ff601e4cbc1]
[bt] (2) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(mxnet::CopyFromTo(mxnet::NDArray const&, mxnet::NDArray const&, int, bool)+0x723) [0x7ff604982623]
[bt] (3) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(mxnet::imperative::PushFComputeEx(std::function<void (nnvm::NodeAttrs const&, mxnet::OpContext const&, std::vector<mxnet::NDArray, std::allocator<mxnet::NDArray> > const&, std::vector<mxnet::OpReqType, std::allocator<mxnet::OpReqType> > const&, std::vector<mxnet::NDArray, std::allocator<mxnet::NDArray> > const&)> const&, n

In [3]:
%%time
if __name__ == "__main__":
    deep_state(seed=42, data="m4_daily", epochs=45, batches=50)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [05:43<00:00,  6.88s/it, avg_epoch_loss=-1.4]    
INFO:root:Epoch[0] Elapsed time 343.905 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-1.396312
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [01:32<00:00,  1.85s/it, avg_epoch_loss=0.0665]
INFO:root:Epoch[1] Elapsed time 92.285 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.066499
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [01:30<00:00,  1.82s/it, avg_epoch_loss=-1.24]
INFO:root:Epoch[2] Elapsed time 90.859 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-1.238061
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [01:32<00:00,  1.85s/it, avg_epoch_loss=-1.52]
INFO:root:Epoch[3] Elapsed time 92.670 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=-1.518090
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [01:30<00:00,  1.82s/it,

{'MASE': 4.00648902,
 'MSIS': 64.1235091,
 'sMAPE': 0.03672628,
 'wQuantileLoss[0.5]': 0.03428623,
 'wQuantileLoss[0.9]': 0.02183124}
CPU times: user 5h 39min 43s, sys: 4min 50s, total: 5h 44min 34s
Wall time: 1h 23min 24s


In [4]:
%%time
if __name__ == "__main__":
    deep_state(seed=43, data="m4_daily", epochs=35, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [05:39<00:00,  6.78s/it, avg_epoch_loss=-1.46]    
INFO:root:Epoch[0] Elapsed time 339.080 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-1.462676
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [01:32<00:00,  1.86s/it, avg_epoch_loss=-.0856]
INFO:root:Epoch[1] Elapsed time 92.756 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=-0.085578
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [01:31<00:00,  1.83s/it, avg_epoch_loss=-1.11]
INFO:root:Epoch[2] Elapsed time 91.746 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-1.112522
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [01:33<00:00,  1.87s/it, avg_epoch_loss=-1.28]
INFO:root:Epoch[3] Elapsed time 93.308 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_los

{'MASE': 3.96658989,
 'MSIS': 62.97621606,
 'sMAPE': 0.03647926,
 'wQuantileLoss[0.5]': 0.03406613,
 'wQuantileLoss[0.9]': 0.02173784}
CPU times: user 4h 39min 19s, sys: 4min 5s, total: 4h 43min 25s
Wall time: 1h 8min 2s


In [5]:
%%time
if __name__ == "__main__":
    deep_state(seed=43, data="m4_daily", epochs=45, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [05:49<00:00,  6.99s/it, avg_epoch_loss=-1.46]    
INFO:root:Epoch[0] Elapsed time 349.533 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-1.462676
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [01:32<00:00,  1.85s/it, avg_epoch_loss=-.0856]
INFO:root:Epoch[1] Elapsed time 92.581 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=-0.085578
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [01:32<00:00,  1.84s/it, avg_epoch_loss=-1.11]
INFO:root:Epoch[2] Elapsed time 92.112 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-1.112522
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [01:33<00:00,  1.86s/it, avg_epoch_loss=-1.28]
INFO:root:Epoch[3] Elapsed time 93.218 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_los

{'MASE': 3.95546677,
 'MSIS': 61.90661119,
 'sMAPE': 0.03626362,
 'wQuantileLoss[0.5]': 0.03392659,
 'wQuantileLoss[0.9]': 0.02124276}
CPU times: user 5h 49min 4s, sys: 4min 51s, total: 5h 53min 56s
Wall time: 1h 24min 13s


In [6]:
%%time
if __name__ == "__main__":
    deep_state(seed=44, data="m4_daily", epochs=35, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [05:48<00:00,  6.97s/it, avg_epoch_loss=-1.44]    
INFO:root:Epoch[0] Elapsed time 348.361 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-1.439810
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [01:32<00:00,  1.85s/it, avg_epoch_loss=0.311]
INFO:root:Epoch[1] Elapsed time 92.620 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.310640
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [01:33<00:00,  1.88s/it, avg_epoch_loss=-1.09]
INFO:root:Epoch[2] Elapsed time 93.873 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-1.089874
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [01:33<00:00,  1.87s/it, avg_epoch_loss=-1.55]
INFO:root:Epoch[3] Elapsed time 93.505 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'

{'MASE': 3.93098797,
 'MSIS': 61.47390487,
 'sMAPE': 0.03601161,
 'wQuantileLoss[0.5]': 0.0335479,
 'wQuantileLoss[0.9]': 0.02094752}
CPU times: user 4h 41min 34s, sys: 4min 7s, total: 4h 45min 42s
Wall time: 1h 8min 38s


In [7]:
%%time
if __name__ == "__main__":
    deep_state(seed=44, data="m4_daily", epochs=45, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [05:40<00:00,  6.81s/it, avg_epoch_loss=-1.44]    
INFO:root:Epoch[0] Elapsed time 340.440 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-1.439810
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [01:32<00:00,  1.86s/it, avg_epoch_loss=0.311]
INFO:root:Epoch[1] Elapsed time 92.955 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.310640
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [01:31<00:00,  1.83s/it, avg_epoch_loss=-1.09]
INFO:root:Epoch[2] Elapsed time 91.617 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-1.089874
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [01:33<00:00,  1.86s/it, avg_epoch_loss=-1.55]
INFO:root:Epoch[3] Elapsed time 93.164 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'

{'MASE': 3.9444068,
 'MSIS': 62.48458484,
 'sMAPE': 0.03599004,
 'wQuantileLoss[0.5]': 0.03369017,
 'wQuantileLoss[0.9]': 0.0211799}
CPU times: user 5h 49min 40s, sys: 4min 52s, total: 5h 54min 33s
Wall time: 1h 24min 17s
